In [2]:
from collections import Counter
import pandas as pd
from pprint import pprint

In [3]:
df_tennis = pd.read_csv('playtennnis.csv')
df_tennis

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [4]:
def entropy(xs:list):
    import math
    return sum([-x * math.log2(x) for x in xs])

In [5]:
def entropy_of_list(list_:list):
    cnt = Counter(x for x in list_)
    n = len(list_)
    probability = [x/n for x in cnt.values()]
    return entropy(probability)

In [14]:
def information_gain(df:pd.DataFrame, split, target, trace=0):
    df_split = df.groupby(split)
    n = len(df)
    df_arg_ent = df_split.agg({target:[entropy_of_list,lambda x:len(x)/n]})[target]
    # print(df_arg_ent)
    df_arg_ent.columns=['Entropy','PropObervation']
    new_entorpy= sum(df_arg_ent['Entropy'] * df_arg_ent['PropObervation'])
    old_entropy = entropy_of_list(df[target])
    return old_entropy-new_entorpy

In [15]:
total_entropy = entropy_of_list(df_tennis['PlayTennis'])
total_entropy

0.9402859586706311

In [18]:
def id3(df:pd.DataFrame, target:str, attribute:list[str], defult_class:str=None):
    cnt = Counter(x for x in df[target])
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attribute):
        return defult_class
    else:
        defult_class = max(cnt.keys())
        # print(type(defult_class), defult_class)
        ganiz = [information_gain(df,attr,target) for attr in attribute]
        index_of_max = ganiz.index(max(ganiz))
        best_attr=attribute[index_of_max]
        tree = {best_attr:{}}
        reaming = [i for i in attribute if i!=best_attr]
        for attr_val,data_subset in df.groupby(best_attr):
            substree = id3(data_subset,target,reaming,defult_class)
            tree[best_attr][attr_val]=substree
        return tree

In [19]:
attribute = list(df_tennis.columns[:])
attribute.remove('PlayTennis')
tree = id3(df_tennis, 'PlayTennis', attribute)

<class 'str'> Yes
<class 'str'> Yes
<class 'str'> Yes


In [38]:
pprint(tree)

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
